In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent, create_react_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv

load_dotenv() 




True

In [2]:
db = SQLDatabase.from_uri("sqlite:////Users/uzermughal/Documents/mvp garmin/garmin_data.db")
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


In [3]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [4]:
sql_agent = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [5]:
sql_agent.invoke('what is my most popular activity and how many have I done of each ')



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:activities_dataI should look at the schema of the activities_data table to see what I can query.
Action: sql_db_schema
Action Input: activities_data
CREATE TABLE activities_data (
	"activityId" INTEGER, 
	"activityName" TEXT, 
	"aerobicTrainingEffect" REAL, 
	"aerobicTrainingEffectMessage" TEXT, 
	"anaerobicTrainingEffect" REAL, 
	"averageHR" REAL, 
	"averageRunningCadenceInStepsPerMinute" REAL, 
	"averageSpeed" REAL, 
	"beginTimestamp" INTEGER, 
	"bmrCalories" REAL, 
	calories REAL, 
	distance REAL, 
	"elevationCorrected" BOOLEAN, 
	"endTimeGMT" TEXT, 
	"hrTimeInZone_1" REAL, 
	"hrTimeInZone_2" REAL, 
	"hrTimeInZone_3" REAL, 
	"hrTimeInZone_4" REAL, 
	"hrTimeInZone_5" REAL, 
	"maxHR" REAL, 
	"maxSpeed" REAL, 
	"moderateIntensityMinutes" INTEGER, 
	"movingDuration" REAL, 
	"startTimeGMT" TEXT, 
	steps INTEGER
)

/*
3 rows from activities_data table:
activityId	activityName	aerobicTrainingEffect	aerobic

{'input': 'what is my most popular activity and how many have I done of each ',
 'output': 'The most popular activity is Strength, and it has been done 12 times. Other activities include Kingston upon Thames Soccer/Football (5), Wandsworth Soccer/Football (3), Lambeth Running (2), and several activities done once.'}

In [6]:
from langchain_tavily import TavilySearch

os.environ['TAVILY_API_KEY'] = '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'
web_search = TavilySearch(max_results=3)




In [17]:
from langchain.prompts import PromptTemplate

prompt = (
        "You are an expert, AI-powered Personal Trainer Agent, designed to help users reach their fitness and health goals. "
        "You work alongside a SQL Data Analyst Agent who retrieves and analyzes structured data from the user's health and fitness database. "
        "You specialize in translating data and insights into actionable, personalized fitness coaching.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with personal trainer related tasks, DO NOT do any analysis\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text.\n\n"
    )


personal_trainer_agent = create_react_agent(
    llm=llm,
    tools=[web_search],
    prompt=prompt,
    # name="personal_trainer_agent",
    )

AttributeError: 'str' object has no attribute 'input_variables'

In [13]:
agent_executor = create_react_agent(llm,prompt, web_search)


AttributeError: 'TavilySearch' object has no attribute 'input_variables'

In [ ]:
from langgraph_supervisor import create_supervisor

supervisor = create_supervisor(
    model=llm,
    agents=[sql_agent, personal_trainer_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

ValueError: Please specify a name when you create your agent, either via `create_react_agent(..., name=agent_name)` or via `graph.compile(name=name)`.

In [ ]:
from IPython.display import display, Image

display(Image(supervisor.get_graph().draw_mermaid_png()))

NameError: name 'supervisor' is not defined